In [9]:
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot
import numpy as np
import plotly.graph_objects as go

In [10]:
file_path = 'USD_BRL Dados Históricos.csv'

# above .data file is comma delimited
dolar_df = pd.read_csv(file_path, sep=",")
dolar_df = dolar_df[::-1].reset_index()
dolar_df = dolar_df.drop(columns=['index'])
dolar_df

,Date,Last,Opening,Max,Min,Var%
0,18.12.1992,"0,0052","0,0052","0,0052","0,0052","-1,89%"
1,21.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
2,22.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
3,23.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
4,24.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
...,...,...,...,...,...,...
7296,29.12.2020,"5,2099","5,2460","5,2473","5,1645","-0,69%"
7297,30.12.2020,"5,1937","5,2051","5,2361","5,1519","-0,31%"
7298,31.12.2020,"5,1935","5,1940","5,1940","5,1940","-0,00%"
7299,01.01.2021,"5,1935","5,1934","5,1935","5,1934","0,00%"


In [17]:
past_features_qt = 5
new_features = [[] for i in range(past_features_qt)]
df_length = len(dolar_df['Last'])
df_step_range = range(past_features_qt, df_length)
for i in df_step_range:
    for j in reversed(range(0, past_features_qt)):
        new_features[j].append(dolar_df['Last'][(i-past_features_qt)+j])
        
features = pd.DataFrame() # dolar_df[['Date', 'Opening', 'Max', 'Min', 'Var%']][past_features_qt:]

for i in range(0, past_features_qt):
    features['t-{}'.format(i+1)] = new_features[past_features_qt-i-1]
    
labels = dolar_df[['Last']][past_features_qt:]

In [18]:
# Aplly Normalization to values of the features
def normalize_features(x):
    return (scale*((x-min_value)/(max_value-min_value)))+new_min

def to_float(x):
    return float(x.replace(',', '.'))

def var_to_float(x):
    x = x.replace(',', '.')
    x = x.replace('%', '')
    return float(x)

def date_to_int(x):
    return int(x.replace('.', ''))
   
for column in features:
    features[column] = features[column].apply(to_float)
    
    new_max = 0.5
    new_min = -0.5
    max_value = features[column].max()
    min_value = features[column].min()
    scale = new_max - new_min
    features[column] = features[column].apply(normalize_features)

labels['Last'] = labels['Last'].apply(to_float)

# features['Var%'] = dolar_df['Var%'].apply(var_to_float)
# features['Var%'] = features['Var%'].apply(normalize_features)

In [19]:
test_percent = 10
train_samples_qt = int((dolar_df.shape[0]-past_features_qt)*((100-test_percent)/100))

x_train = features[:][:train_samples_qt]
y_train = labels[:][:train_samples_qt]
x_test = features[:][train_samples_qt:]
y_test = labels[:][train_samples_qt:]

input_shape = (features.shape[1],)
features

,t-1,t-2,t-3,t-4,t-5
0,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000
1,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000
2,-0.499983,-0.500000,-0.500000,-0.500000,-0.500000
3,-0.499983,-0.499983,-0.500000,-0.500000,-0.500000
4,-0.499949,-0.499983,-0.499983,-0.500000,-0.500000
...,...,...,...,...,...
7291,0.391037,0.386004,0.385987,0.376347,0.369972
7292,0.384882,0.391037,0.386004,0.385987,0.376347
7293,0.382128,0.384882,0.391037,0.386004,0.385987
7294,0.382094,0.382128,0.384882,0.391037,0.386004


In [20]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu", input_shape=input_shape),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                384       
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 4,609
Trainable params: 4,609
Non-trainable params: 0
_________________________________________________________________


In [25]:
batch_size = 125
epochs = 25

model.compile(loss="mean_squared_error", optimizer="RMSprop", metrics=['mae', 'mse'])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/25
48/48 [==============================] - 0s 4ms/step - loss: 0.0073 - mae: 0.0440 - mse: 0.0073 - val_loss: 0.0020 - val_mae: 0.0340 - val_mse: 0.0020
Epoch 2/25
48/48 [==============================] - 0s 2ms/step - loss: 0.0025 - mae: 0.0401 - mse: 0.0025 - val_loss: 0.0017 - val_mae: 0.0301 - val_mse: 0.0017
Epoch 3/25
48/48 [==============================] - 0s 2ms/step - loss: 0.0025 - mae: 0.0426 - mse: 0.0025 - val_loss: 0.0032 - val_mae: 0.0465 - val_mse: 0.0032
Epoch 4/25
48/48 [==============================] - 0s 1ms/step - loss: 0.0025 - mae: 0.0427 - mse: 0.0025 - val_loss: 0.0031 - val_mae: 0.0468 - val_mse: 0.0031
Epoch 5/25
48/48 [==============================] - 0s 2ms/step - loss: 0.0028 - mae: 0.0463 - mse: 0.0028 - val_loss: 0.0027 - val_mae: 0.0410 - val_mse: 0.0027
Epoch 6/25
48/48 [==============================] - 0s 2ms/step - loss: 0.0024 - mae: 0.0409 - mse: 0.0024 - val_loss: 0.0048 - val_mae: 0.0601 - val_mse: 0.0048
Epoch 7/25
48/48 [==========

In [30]:
score = model.evaluate(x_test, y_test, verbose=0)

predictions = model.predict(x_test)

data = []
trace = go.Scatter(x=dolar_df['Date'][(train_samples_qt+past_features_qt):], 
                   y=np.reshape(predictions, -1),
                   marker=dict(color='red'),
                   name="Predição")
data.append(trace)

trace = go.Scatter(x=dolar_df['Date'][(train_samples_qt+past_features_qt):], 
                   y=y_test['Last'],
                   marker=dict(color='blue'),
                   name="Rótulos")
data.append(trace)

layout =  go.Layout(xaxis={'title': 'Tempo'},
                    yaxis={'title': 'Valor'},
                    margin={'l': 40, 'b': 40, 't': 50, 'r': 50},
                    hovermode='closest')

figure = go.Figure(data=data, layout=layout)  
figure.show()